# Hands-On Machine Learning with the MNIST dataset.
Artificial Intelligence (AI) encompasses a broad range of concepts and technologies aimed at creating systems capable of performing tasks that typically require human intelligence. 

One of the core components of AI is neural networks, which are computational models inspired by the human brain's structure and function. Neural networks consist of interconnected layers of nodes, or neurons, that process and transmit information. These networks are particularly effective in recognizing patterns and making predictions. A common application of neural networks is in image recognition, where the MNIST dataset serves as a benchmark. The MNIST dataset contains a large collection of handwritten digits, which are used to train and evaluate the performance of neural networks in accurately classifying and recognizing digit images. This dataset has become a standard for testing new algorithms and models in the field of machine learning and AI.

## Import Statements

In [7]:
import random
import matplotlib.pyplot as plt
import numpy as np

from keras.datasets import mnist # MNIST dataset, included in Keras
from keras.models import Sequential # type of model we will be using

from keras.layers import Dense, Dropout, Activation, Flatten # types of layers we will be using
from keras.utils import to_categorical # utilities for one-hot encoding of ground truth values

# MNIST Dataset
The MNIST dataset is a widely used benchmark in the field of machine learning and artificial intelligence, particularly for image recognition tasks. It consists of 70,000 grayscale images of handwritten digits, each with dimensions of 28x28 pixels. The dataset is divided into 60,000 training images and 10,000 test images, providing a comprehensive set of examples for training and evaluating neural network models. The simplicity and standardized format of the MNIST dataset make it an ideal starting point for developing and testing new algorithms and models in the realm of digit classification.

Now let's load in the MNIST dataset from Keras and visualize some of the samples.

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Display the first 10 images
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(X_train[i], cmap='gray', interpolation='none')
    plt.title("Label: {}".format(y_train[i]))

In [ ]:
# TODO Examine the shapes of X_train, y_train, X_test, y_test 
# Hint: print the shape attribute of each numpy array


# Formatting our Input
Instead of a 28 x 28 matrix, we build our network to accept a 784-length vector.

Each image needs to be then reshaped (or flattened) into a vector. We'll also normalize the inputs to be in the range [0-1] rather than [0-255]. Normalizing inputs is generally recommended, so that any additional dimensions (for other network architectures) are of the same scale.

![flatten.png](images/flatten.png)

In [ ]:
# TODO Reshape the X_train and X_test data to be 784-length vectors
# Hint: Use the reshape method of numpy arrays, with the arguments (length of X_train or X_test, 784)

# Given -> We'll change the integers into 32-bit floating point nums
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# TODO Normalize each value in X_train and X_test to be between 0 and 1 by dividing by 255
# Hint: Divide each numpy array by 255

We then want to modify our classes (unique digits) to be in the one-hot format, i.e.

```
0 -> [1, 0, 0, 0, 0, 0, 0, 0, 0]
1 -> [0, 1, 0, 0, 0, 0, 0, 0, 0]
2 -> [0, 0, 1, 0, 0, 0, 0, 0, 0]
etc.
```

If the final output of our network is very close to one of these classes, then it is most likely that class. For example, if the final output is:

```
[0, 0.94, 0, 0, 0, 0, 0.06, 0, 0]
```
then it is most probable that the image is that of the digit `1`.

In [ ]:
num_classes = 10

# TODO Convert the labels from digits to one-hot-encoded vectors
# Hint: Use the to_categorical function from keras.utils with the arguments (y_train or y_test, num_classes)

# Building a Model
We'll be using the Keras framework to build a Multi-layer Perceptron consisting of one hidden layer. We'll start by setting up the model.

In [ ]:
# TODO create the model using the Sequential class
# Hint: Use the Sequential class constructor

## The first hidden layer

Now we'll add a hidden layer of size 32. This will be a Dense layer, practically a set of 32 nodes (artifical neurons). Each node receives an element from each input vector, and then applies some weight and bias to it. Each of these layers will need an activation function, which is a non-linear function applied to the output of a layer. For our hidden layer, we will be using ReLU as the activation function.

ReLU is defined as: $$f(x) = max(0, x)$$

![relu.jpg](images/relu.jpg)

In [ ]:
# TODO Add a Dense layer with 32 nodes to the model
# Hint: Use the add method of the model with the arguments Dense(32, input_shape=(784,)) 
#       *** (784,) as a shape indicates the 784-length vector of our input

# TODO Add a ReLU activation layer to the model
# Hint: Use the add method of the model with the argument Activation('relu')

# OPTIONAL: Add a Dropout layer to the model
# Dropout layers are used to prevent overfitting by randomly setting some of the weights to 0 during training
# Hint: Use the add method of the model with the argument Dropout(some probability)

In [1]:
# OPTIONAL: Add another Dense layer with some amount of nodes
#           Don't forget the activation function and dropout layer if you want to use dropout
# Hint: Do the same as the first Dense layer, but with a different number of nodes/dropout

# The Final Output Layer

The output layer of a model for classification will be equal to the number of classes (10 in our case). We first need a dense layer to connect to the previous fully-connected layer. Then, we apply a softmax activation function, which represents the output in terms of a probability distribution for each of the classes. All of the values output from the softmax are non-negative and sum to 1.

The output of the softmax would be something like what we saw earlier:

```
[0, 0.94, 0, 0, 0, 0, 0.06, 0, 0]
```

where each value indicates the probability of the input being in each class according to the model.

In [ ]:
# TODO Add a Dense layer with 10 nodes to the model
# Hint: Use the add method of the model specifying Dense(10)

# TODO Add a softmax activation layer to the model
# Hint: Use the add method of the model with the argument Activation('softmax')

# TODO Visualize the model using the summary method of the model

# Compiling and Training the Model

Keras is built on top of Theano and TensorFlow. Both packages allow you to define a *computation graph* in Python, which then compiles and runs efficiently on the CPU or GPU without the overhead of the Python interpreter.

When compiing a model, Keras asks you to specify your **loss function** and your **optimizer**. The loss function we'll use here is called *categorical cross-entropy*, and is a loss function well-suited to comparing two probability distributions.

Our predictions are probability distributions across the ten different digits (e.g. "we're 80% confident this image is a 3, 10% sure it's an 8, 5% it's a 2, etc."), and the target is a probability distribution with 100% for the correct category, and 0 for everything else. The cross-entropy is a measure of how different your predicted distribution is from the target distribution. [More detail at Wikipedia](https://en.wikipedia.org/wiki/Cross_entropy)

The optimizer helps determine how quickly the model learns through **gradient descent**. The rate at which descends a gradient is called the **learning rate**.

![gradient_descent.jpg](images/gradient_descent.png)

![learning_rate.jpg](images/learning_rate.png)

It might seem that smaller learning rates are better. However, it's important to not get stuck in local minima, which does not allow models to converge. This may call for a larger learning rate to "jump" out of the local minimum.

![comp](images/complicated_loss_function.png)

In [ ]:
# TODO We are going to compile the model using the Adam optimizer, categorical crossentropy loss, and accuracy as a metric
# Hint: Use the compile method of the model with the arguments (optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Now let's train the model!

We will pick a batch size to train the model. 

The batch size determines over how much data per step is used to compute the loss function, gradients, and back propagation. Large batch sizes allow the network to complete it's training faster; however, there are other factors beyond training speed to consider.

Too large of a batch size smoothes the local minima of the loss function, causing the optimizer to settle in one because it thinks it found the global minimum.

Too small of a batch size creates a very noisy loss function, and the optimizer may never find the global minimum.

So a good batch size may take some trial and error to find!


In [ ]:
# TODO Train the model with a batch_size of 128 for 10 epochs and a verbose level of 1
# Hint: Use the fit method of the model with the arguments (X_train, y_train, batch_size=128, epochs=10, verbose=1)

# Evaluating on Test Data

So we've trained the model on data it sees during every epoch during training. How about on data it has not seen yet?

In [ ]:
# TODO Evaluate the model on the test data
# Hint: Create a score variable and set it equal to the output of
# the evaluate method of the model with the arguments (X_test, y_test)

print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

# Sanity Checks/Inspecting Output

It's always a good idea to inspect the output and make sure everything looks sane. Here we'll look at some examples it gets right, and some examples it gets wrong.

In [ ]:
# The predict_classes function outputs the highest probability class
# according to the trained classifier for each input example.
predicted_classes = model.predict_classes(X_test)

# Check which items we got right / wrong
correct_indices = np.nonzero(predicted_classes == y_test)[0]

incorrect_indices = np.nonzero(predicted_classes != y_test)[0]

In [ ]:
plt.figure()
for i, correct in enumerate(correct_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[correct].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[correct], y_test[correct]))
    
plt.tight_layout()
    
plt.figure()
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[incorrect].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[incorrect], y_test[incorrect]))
    
plt.tight_layout()

# CIFAR-10 and CNNs

Now that we've trained a simple model on MNIST, how does that extend to other datasets, and to other types of models like Convolutional Neural Networks?

In the cells below, I'll walk through the setup, training, and testing of a simple MLP and CNN on the CIFAR-10 dataset. There are no TODOs in here, so you can run each cell sequentially without having to write the code yourself. The goal of this section is to explore how this simple example can extend further to more complicated problems.

The CIFAR-10 dataset is a collection of 60,000 32x32 color images in 10 different classes, with 6,000 images per class. The dataset is divided into 50,000 training images and 10,000 test images. The classes represent common objects such as airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, and trucks. This dataset is widely used for training machine learning and computer vision algorithms, and it presents a more complex challenge compared to the MNIST dataset due to the variability in object appearances and backgrounds.

In [ ]:
from keras.datasets import cifar10

# Loading the CIFAR-10 dataset in

These images are (32, 32, 3), with the third dimension being the color channels for R G and B. For this type of network, I still do need to flatten them.

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 32 * 32 * 3)
X_test = X_test.reshape(X_test.shape[0], 32 * 32 * 3)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

In [8]:
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [ ]:
model = Sequential()
model.add(Dense(512, input_shape=(3072,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(10))
model.add(Activation('softmax'))

model.summary()

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=128, epochs=5, verbose=1)

In [ ]:
score = model.evaluate(X_test, y_test)
print('Test score:', score[0])
print('Test accuracy:', score[1])